In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
%matplotlib inline
from tensorflow import keras



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
data = []

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        data.append(os.path.join(dirname, filename))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2024-07-26 00:38:15.431286: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-26 00:38:15.431445: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-26 00:38:15.606478: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/playground-series-s4e7/sample_submission.csv
/kaggle/input/playground-series-s4e7/train.csv
/kaggle/input/playground-series-s4e7/test.csv


In [2]:
df_train = pd.read_csv(data[1])
df_test = pd.read_csv(data[2])
df_train = df_train.drop(columns =['id'])
df_test= df_test.drop(columns= ['id'])

In [3]:
binary_features_names= ['Previously_Insured','Driving_License','Gender','Vehicle_Damage']
cont_features_names = ['Age','Annual_Premium','Vintage']
ordinal_features_names= ['Region_Code','Vehicle_Age','Policy_Sales_Channel']


In [4]:
from sklearn import preprocessing
train_model = df_train.copy()
test_model = df_test.copy()



oe = preprocessing.OrdinalEncoder()
train_model[ordinal_features_names] = oe.fit_transform(train_model.filter(ordinal_features_names)).astype('int')
train_model[binary_features_names] = oe.fit_transform(train_model.filter(binary_features_names)).astype('int')
train_model = pd.get_dummies(train_model,columns = binary_features_names ).astype('int')

test_model[ordinal_features_names] = oe.fit_transform(test_model.filter(ordinal_features_names)).astype('int')
test_model[binary_features_names] = oe.fit_transform(test_model.filter(binary_features_names)).astype('int')
test_model = pd.get_dummies(test_model,columns = binary_features_names ).astype('int')


In [5]:
from sklearn.model_selection import train_test_split

X = train_model.loc[:, train_model.columns != "Response"]
y = train_model['Response']


X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.005, random_state=42)

print("Train set size:", X_train.shape)
print("Dev set size:", X_dev.shape)


Train set size: (11447274, 14)
Dev set size: (57524, 14)


In [6]:
import pprint
from functools import partial
import warnings
warnings.filterwarnings("ignore")
from xgboost import XGBClassifier
from time import time
import xgboost as xgb
from sklearn.model_selection import KFold

from skopt import BayesSearchCV
from skopt.callbacks import DeadlineStopper, DeltaYStopper
from skopt.space import Real, Categorical, Integer
from sklearn.metrics import make_scorer, log_loss



In [7]:
# Reporting util for different optimizers
def report_perf(optimizer, X, y, title, callbacks=None):
    """
    A wrapper for measuring time and performances of different optmizers
    
    optimizer = a sklearn or a skopt optimizer
    X = the training set 
    y = our target
    title = a string label for the experiment
    """
    start = time()
    if callbacks:
        optimizer.fit(X, y, callback=callbacks)
    else:
        optimizer.fit(X, y)
    best_score = optimizer.best_score_
    best_score_std = optimizer.cv_results_['std_test_score'][optimizer.best_index_]
    best_params = optimizer.best_params_
    print((title + " took %.2f seconds,  candidates checked: %d, best CV score: %.3f "
           +u"\u00B1"+" %.3f") % (time() - start, 
                                  len(optimizer.cv_results_['params']),
                                  best_score,
                                  best_score_std))    
    print('Best parameters:')
    pprint.pprint(best_params)
    print()
    return best_params

In [8]:
neg_log_loss_scorer = make_scorer(log_loss, greater_is_better=False)
kf = KFold(n_splits = 5, shuffle = True, random_state= 0)

reg = XGBClassifier(
        objective='binary:logistic',
        eval_metric='auc',

        verbosity=3,  # set to 0 for no output during training
        n_jobs = -1,)

search_spaces = {
    'learning_rate': Real(0.01, 0.1, 'log-uniform'),  # 작은 범위에서의 지수적 변화
    'n_estimators': Integer(1000, 10000),  # 트리 개수
    'max_depth': Integer(5, 16),  # 최대 깊이
    'reg_lambda': Real(0.1, 100, 'log-uniform'),  # L2 정규화
    'subsample': Real(0.01, 1, 'uniform'),  # 샘플링 비율 (명시적으로 uniform)
    'colsample_bytree': Real(0.1, 1, 'uniform'),  # 피처 샘플링 비율
    'min_child_weight': Real(0.01, 10.0, 'uniform'),  # 최소 자식 노드 가중치
    'max_bin': Integer(32, 512),  # 최대 빈 개수
    'reg_alpha': Real(0.1, 100, 'log-uniform'),  # L1 정규화
    'scale_pos_weight': Real(1e-6, 500, 'log-uniform')  # 불균형 데이터 가중치 조정
}


opt= BayesSearchCV(estimator = reg,
                  search_spaces =search_spaces,
                  scoring =neg_log_loss_scorer,
                  cv = kf,
                  n_iter = 60,
                  n_points = 3,
                  n_jobs = -1,
                  return_train_score = False,
                  refit = False)

In [ ]:
overdone_control = DeltaYStopper(delta = 0.0001)
time_limit_control = DeadlineStopper (total_time = 60*60*6)

best_params = report_perf(opt,X,y,'Xgboost',callbacks= [overdone_control,time_limit_control])